In [ ]:
import glob as glob

import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def read_vars(fName, vName, level_index):
    ds = xr.open_dataset(fName)
#    ds = ds.assign_coords({'geolon': ds['geolon'],
#                           'geolat': ds['geolat']})
    var = ds[vName].isel(z_l=level_index)
    return var

In [ ]:
def plot_var(out_dir, PDY, var, expName, varName, varUnits, cMin, cMax, cMap):
    fig=plt.figure(figsize=(8, 6), clear=True, dpi=60)
    ax = fig.add_subplot(111, projection=ccrs.Robinson())
    im=ax.pcolormesh(var['xh'], var['yh'], var.squeeze(),
                     vmin=cMin, vmax=cMax, cmap=cMap,
                     transform=ccrs.PlateCarree())
    
    cbar=fig.colorbar(im, ax=ax, orientation='horizontal')
    ax.set_title(f'{expName} {PDY}')
    cbar.ax.set_title(f'{varName} [{varUnits}]')
    figName = out_dir + varName+'_'+PDY+'_'+expName+'.png'
    plt.savefig(figName, bbox_inches='tight')
    print(f'Saved {figName}')
    plt.close('all')

In [ ]:
varUnits={
    'Temp':'degC',
    'Salt':'PSU'}

varCbar={
    'Temp':[-2, 32, 'gist_ncar'],
    'Salt':[32, 40, 'nipy_spectral']}

## Set inputs:
- Names of experiments.
- Path to data.
- Dates (yyyymmdd) and hours: `yyyymmddThh`

In [ ]:
data_rootPath = "/scratch3/NCEPDEV/climate/Jiande.Wang/working/scratch/C384-v17/OCEAN/"
expNames = ["B1", "B2", "B3"]

# set date 
#dateStr = "20250101" # start date
#hrStr = "06" # starting hour (on start date)
dateStr = "20250126" # end date
hrStr = "00" # ending hour (on end date)

# set (3-d) variable names and level
vNames = ["Temp", "Salt"]
level_index = 0 # level-1 (aka "k-index")

PDY = dateStr + "T" + hrStr

# path where figures will be saved
out_dir = "/home/Santha.Akella/for_folks/JiandeW/Sep2025/figs/"

## Load datasets from all:
- The experiments set in `expNames`.
- Dates.
- Plot user requested variables.

In [ ]:
for iexp, expName in enumerate(expNames):
    print(f"Processing data from:\t{expName}")
    
    dPath = data_rootPath+f"{expName}/gdas.{dateStr}/{hrStr}/model/ocean/history/"
    fName = sorted(glob.glob(dPath + f"gdas.ocean.t*z.inst.f000.nc"))[0]
    print(f"Reading:\t{fName}")

    for iVar, varName in enumerate(vNames):
        var = read_vars(fName, varName, level_index)
        plot_var(out_dir, PDY, var, expName, varName, varUnits[varName], 
                 varCbar[varName][0], varCbar[varName][1], varCbar[varName][2])